
```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 
                                                                                        

# Demo 4 - Files to MSSQL

## Import severals CSV files to one MSSQL Table

FastTransfer use an embedded DuckDB that allow to read CSV, JSON, Parquet files using SQL to read data.
DuckDB infer the source CSVs structure using the first 100_000 rows if no format is provided.
It is possible to give or force a format. Use the [DuckDB CSV import documentation](https://duckdb.org/docs/stable/data/csv/overview) to see what is possible


### using Datadriven and filenames as distributedkeycolumn

DuckDB allow to retrieve data from CSV as a sql query that FastTransfer will understand and push to a target (MSSQL but also PostgreSQL, Oracle, MySQL, SAP HANA, Clickhouse). This allow **FastTransfer to become an universal CSV parallel loader**. For that you must use sourceconnectiontype as **duckdbstream**

Adding this to the automapping method using column names and you have the perfect import tool !

In [14]:
ls D:\temp\mssql\*orders*.csv | select name, length

key,type,value
Name,System.String,mssql_orders_chunck_000.csv
Length,System.Int64,201817934


key,type,value
Name,System.String,mssql_orders_chunck_001.csv
Length,System.Int64,202830933


key,type,value
Name,System.String,mssql_orders_chunck_002.csv
Length,System.Int64,203716329


key,type,value
Name,System.String,mssql_orders_chunck_003.csv
Length,System.Int64,203691780


key,type,value
Name,System.String,mssql_orders_chunck_004.csv
Length,System.Int64,203736826


key,type,value
Name,System.String,mssql_orders_chunck_005.csv
Length,System.Int64,203742119


key,type,value
Name,System.String,mssql_orders_chunck_006.csv
Length,System.Int64,203732078


key,type,value
Name,System.String,mssql_orders_chunck_007.csv
Length,System.Int64,203733178


key,type,value
Name,System.String,mssql_orders_chunck_008.csv
Length,System.Int64,203733773


key,type,value
Name,System.String,mssql_orders_chunck_009.csv
Length,System.Int64,204134293


In [18]:
duckdb.exe -c "SELECT * FROM read_csv('D:\temp\mssql\*orders*.csv', filename=true);"

┌─────────────┬────────────┬───────────┬───┬──────────────┬──────────────────────┬──────────────────────┐
│ o_orderdate │ o_orderkey │ o_custkey │ … │ o_totalprice │      o_comment       │       filename       │
│    date     │   int64    │   int64   │   │   varchar    │       varchar        │       varchar        │
├─────────────┼────────────┼───────────┼───┼──────────────┼──────────────────────┼──────────────────────┤
│ 1998-06-16  │    3381152 │     10531 │ … │ 72696,33     │ dolphins wake slyl…  │ D:\temp\mssql\mssq…  │
│ 1998-06-16  │    6187044 │     15350 │ … │ 162758,15    │ ironic requests ha…  │ D:\temp\mssql\mssq…  │
│ 1998-06-16  │     970658 │     30523 │ … │ 118376,13    │ slyly regular idea…  │ D:\temp\mssql\mssq…  │
│ 1998-06-16  │     554245 │     58189 │ … │ 332965,55    │ blithely ironic ex…  │ D:\temp\mssql\mssq…  │
│ 1998-06-16  │    4016673 │     71665 │ … │ 37324,46     │ carefully regular …  │ D:\temp\mssql\mssq…  │
│ 1998-06-16  │    2248453 │     77912 │ … │ 2

In [19]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceserver ":memory:" `
--query "SELECT * exclude filename FROM read_csv('D:\temp\mssql\*orders*.csv', filename=true)" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
--method "DataDriven" `
--distributeKeyColumn "filename" `
--datadrivenquery "select file from glob('D:\temp\mssql\*orders*.csv')" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 16 `
--runid "CSV_To_MSSQL_Parallel16_DataDriven_filename"

2025-09-23T00:39:23.052 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (27 day(s) left).
2025-09-23T00:39:23.061 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-23T00:39:23.061 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-23T00:39:23.061 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-23T00:39:23.061 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 100080
2025-09-23T00:3

#### You can also import in parallel one big CSV file. It will be slower for sure but faster than a single threaded import.

In [27]:
duckdb.exe -c "SELECT * FROM read_csv('D:\temp\PGCOPY_ONEBIG\PGCOPY_ORDERS.csv');"

┌────────────┬───────────┬───────────────┬───┬─────────────────┬────────────────┬──────────────────────┐
│ o_orderkey │ o_custkey │ o_orderstatus │ … │     o_clerk     │ o_shippriority │      o_comment       │
│   int64    │   int64   │    varchar    │   │     varchar     │     int64      │       varchar        │
├────────────┼───────────┼───────────────┼───┼─────────────────┼────────────────┼──────────────────────┤
│          1 │    369001 │ O             │ … │ Clerk#000009506 │              0 │ ly express platele…  │
│          2 │    780017 │ O             │ … │ Clerk#000008792 │              0 │ ve the furiously f…  │
│          3 │   1233140 │ F             │ … │ Clerk#000009543 │              0 │  after the asympto…  │
│          4 │   1367761 │ O             │ … │ Clerk#000001234 │              0 │ st the furiously b…  │
│          5 │    444848 │ F             │ … │ Clerk#000009248 │              0 │ onic requests. car…  │
│          6 │    556222 │ F             │ … │ Clerk#00

In [34]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceconnectstring "DataSource=:memory:;memory_limit=1GB" `
--query "SELECT * FROM read_csv('D:\temp\PGCOPY_ONEBIG\PGCOPY_ORDERS.csv')" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
--method "None" `
--loadmode "Truncate" `
--mapmethod "Name" `
--runid "CSV_To_MSSQL_NoParallel"

2025-09-23T01:10:06.405 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (27 day(s) left).
2025-09-23T01:10:06.414 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-23T01:10:06.415 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-23T01:10:06.415 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-23T01:10:06.415 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 19136
2025-09-23T01:10:06.415 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_NoParallel -|- INFORMATION -|- tpch_test.dbo.orders

In [35]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceconnectstring "DataSource=:memory:;memory_limit=1GB" `
--query "SELECT * FROM read_csv('D:\temp\PGCOPY_ONEBIG\PGCOPY_ORDERS.csv')" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
--method "RangeId" `
--distributeKeyColumn "o_orderkey" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 4 `
--runid "CSV_To_MSSQL_Parallel_RangeId_4"

2025-09-23T01:10:50.990 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_RangeId_4 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (27 day(s) left).
2025-09-23T01:10:50.999 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_RangeId_4 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-23T01:10:50.999 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_RangeId_4 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-23T01:10:50.999 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_RangeId_4 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-23T01:10:51.000 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_RangeId_4 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 13808
2025-09-23T01:10:51.000 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel_Ra

## Import severals Parquet files to one MSSQL Table

Parquet is a great file format and instead of CSV, the structure of the table inside the parquet file is embedded in the parquet file(s) itself. FastTransfer will use DuckDB under the hood to read parquet data using SQL and will push the data to the target table.

In [47]:
duckdb.exe -c "SELECT count(*) totalrows_orders_sf100_from_parquet_files FROM read_parquet('D:\tpchgen-cli\sf100\*\orders*.parquet')"

┌───────────────────────────────────────────┐
│ totalrows_orders_sf100_from_parquet_files │
│                   int64                   │
├───────────────────────────────────────────┤
│                                 150000000 │
└───────────────────────────────────────────┘


In [48]:
duckdb.exe -c "select file from glob('D:\tpchgen-cli\sf100\*\orders*.parquet')"

┌───────────────────────────────────────────────┐
│                     file                      │
│                    varchar                    │
├───────────────────────────────────────────────┤
│ D:\tpchgen-cli\sf100\orders\orders.1.parquet  │
│ D:\tpchgen-cli\sf100\orders\orders.10.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.11.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.12.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.13.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.14.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.15.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.16.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.17.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.18.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.19.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.2.parquet  │
│ D:\tpchgen-cli\sf100\orders\orders.20.parquet │
│ D:\tpchgen-cli\sf100\orders\orders.3.parquet  │
│ D:\tpchgen-cli\sf100\orders\orders.4.parquet  │
│ D:\tpchgen-cli\sf100\orders\orders.5.parquet  │


In [6]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceserver ":memory:" `
--query "SELECT * exclude filename FROM read_parquet('D:\tpchgen-cli\sf100\*\orders*.parquet', filename=true)" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders" `
--method "DataDriven" `
--distributeKeyColumn "filename" `
--datadrivenquery "select file from glob('D:\tpchgen-cli\sf100\*\orders*.parquet')" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 12 `
--runid "PARQUET_To_MSSQL_Parallel12_DataDriven_filename"

2025-09-01T22:47:45.512 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer - running in trial mode – will expires on 2025‑09‑19 (17 day(s) left).
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- Starting
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- Process ID : 38492
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- 

This is generally the fastest import format. To Generate parquet file from databases you can try [FastBCP](https://www.arpe.io/fastbcp/)